# Chapter 7. 정규표현식

## 7-1. 정규 표현식 살펴보기
정규 표현식은 복잡한 문자열을 처리할 때 사용하는 기법으로 문자열을 처리하는 모든 곳에서 사용한다.
> 정규표현식은 줄여서 간단히 "정규식"이라고도 말한다.

##### 정규표현식은 왜 필요한가?

`주민등록번호를 포함하고 있는 텍스트가 있다. 이 텍스트에 포함된 모든 주민등록번호의 뒷자리를 * 문자로 변경해 보자.`

In [1]:
data = """
park 800905-1049118
kim  700905-1059119
"""

result = []
for line in data.split("\n"):
    word_result = []
    for word in line.split(" "):
        if len(word) == 14 and word[:6].isdigit() and word[7:].isdigit():
            word = word[:6] + "-" + "*******"
        word_result.append(word)
    result.append(" ".join(word_result))
print("\n".join(result))


park 800905-*******
kim  700905-*******



In [2]:
# 정규표현식 사용
import re 

data = """
park 800905-1049118
kim  700905-1059119
"""

pat = re.compile("(\d{6})[-]\d{7}")
print(pat.sub("\g<1>-*******", data))


park 800905-*******
kim  700905-*******



## 7-2. 정규 표현식 시작하기

##### 정규 표현식의 기초, 메타 문자
> 메타 문자란 원래 그 문자가 가진 뜻이 아닌 특별한 용도로 사용하는 문자를 말한다.
```
. ^ $ * + ? { } [ ] \ | ( )
```

**문자 클래스 []**
  
> 문자 클래스를 만드는 메타 문자인 [] 사이에는 어떤 문자도 들어갈 수 있다. 
  
   
> [a-zA-Z] : 알파벳 모두  
[0-9] : 숫자
  
`^` 메타 문자를 사용할 경우에는 반대라는 의미를 갖는다. (예를 들어 `[^0-9]`라는 정규 표현식은 숫자가 아닌 문자만 매치된다.)




> **[자주 사용하는 문자 클래스]**  
  
  
[0-9] 또는 [a-zA-Z] 등은 무척 자주 사용하는 정규 표현식이다. 이렇게 자주 사용하는 정규식은 별도의 표기법으로 표현할 수 있다.  

`\d` - 숫자와 매치, `[0-9]`와 동일한 표현식이다.  
`\D` - 숫자가 아닌 것과 매치, `[^0-9]`와 동일한 표현식이다.  
`\s` - whitespace 문자와 매치, `[ \t\n\r\f\v]`와 동일한 표현식이다. 맨 앞의 빈 칸은 공백문자(space)를 의미한다.  
`\S` - whitespace 문자가 아닌 것과 매치, `[^ \t\n\r\f\v]`와 동일한 표현식이다.  
`\w` - 문자+숫자(alphanumeric)와 매치, `[a-zA-Z0-9_]`와 동일한 표현식이다.  
`\W` - 문자+숫자(alphanumeric)가 아닌 문자와 매치, `[^a-zA-Z0-9_]`와 동일한 표현식이다.  

**Dot(.)**  
줄바꿈 문자인 `\n`을 제외한 모든 문자와 매치됨을 의미한다.  
`a.b`는 `a와 b문자 사이에 어떤 문자가 들어가도 모두 매치된다는 의미`이다.  
`a[.]b`는 "a.b" (문자 `.` 그대로) 문자열과 매치되고, "a0b"문자열과는 매치되지 않는다.

**반복(*)**  
바로 앞에 있는 문자가 0부터 무한대(2억개 정도)로 반복될 수 있다.  

정규식|	문자열|Match 여부|	설명
------|------|----------|-------
ca*t  |	ct	  |Yes	    |"a"가 0번 반복되어 매치
ca*t  |	cat	  |Yes	    |"a"가 0번 이상 반복되어 매치 (1번 반복)
ca*t  |	caaat |Yes	    |"a"가 0번 이상 반복되어 매치 (3번 반복)

**반복(+)**  
`+`는 최소 1번 이상 반복될 때 사용한다.

정규식|	문자열|	Match 여부|	설명
-----|-------|-----------|-------
ca+t | 	ct   |	No	     |"a"가 0번 반복되어 매치되지 않음
ca+t |	cat	 | Yes	     |"a"가 1번 이상 반복되어 매치 (1번 반복)
ca+t |	caaat| Yes	     |"a"가 1번 이상 반복되어 매치 (3번 반복)

**반복({m,n}, ?)**  
{} 메타 문자를 사용해 반복 횟수 고정  
{m,n} 은 반복 횟수를 m부터 n까지 매치하며, m 또는 n을 생략할 수도 있다.

1. {m}  
반드시 m번 반복해야할 것을 의미한다.
2. {m,n}  
3. `?`  
`?`메타 문자가 의미하는 것은 `{0,1}`이다. (있어도 되고 없어도 된다.)


##### 파이썬에서 정규 표현식을 지원하는 re 모듈

In [1]:
import re
p = re.compile('ab*') #컴파일

##### 정규식을 이용한 문자열 검색
컴파일된 패턴 객체를 사용해 문자열 검색을 수행한다.  
패턴 객체는 4가지 메서드를 제공한다.
Method  |	목적
--------|-------
match()	|문자열의 처음부터 정규식과 매치되는지 조사한다.
search()|	문자열 전체를 검색하여 정규식과 매치되는지 조사한다.
findall()|	정규식과 매치되는 모든 문자열(substring)을 리스트로 돌려준다.
finditer()|	정규식과 매치되는 모든 문자열(substring)을 반복 가능한 객체로 돌려준다.

In [2]:
import re
p = re.compile('[a-z]+')

In [3]:
# match
m = p.match("python")
print(m)

<re.Match object; span=(0, 6), match='python'>


In [4]:
m = p.match("3 python")
print(m)

None


In [5]:
# 일반적인 활용법

# p = re.compile(정규표현식)
# m = p.match( 'string goes here' )
# if m:
#     print('Match found: ', m.group())
# else:
#     print('No match')

In [7]:
# search
m = p.search("python")
print(m)

# match 메서드를 수행했을 때와 동일하게 매치

<re.Match object; span=(0, 6), match='python'>


In [9]:
m = p.search("3 python")
print(m)

# '3' 이후 문자열과 매치

<re.Match object; span=(2, 8), match='python'>


In [10]:
# findall
result = p.findall("life is too short")
print(result)

['life', 'is', 'too', 'short']


In [11]:
# finditer
result = p.finditer("life is too short")
print(result)

In [12]:
for r in result: print(r)

<re.Match object; span=(0, 4), match='life'>
<re.Match object; span=(5, 7), match='is'>
<re.Match object; span=(8, 11), match='too'>
<re.Match object; span=(12, 17), match='short'>


##### match 객체의 매서드
method |	목적
------|-------
group()|	매치된 문자열을 돌려준다.
start()|	매치된 문자열의 시작 위치를 돌려준다.
end()  |매치된 문자열의 끝 위치를 돌려준다.
span() |	매치된 문자열의 (시작, 끝)에 해당하는 튜플을 돌려준다.

In [13]:
m = p.match("python")
m.group()

'python'

In [17]:
m.start()
# 항상 0일 수 밖에 없다(문자열의 시작부터 조사하기 때문)

0

In [15]:
m.end()

6

In [16]:
m.span()

(0, 6)

In [18]:
m = p.search("3 python")
m.start()

2

> **모듈 단위로 수행하기**  
```python
>>> p = re.compile('[a-z]+')
>>> m = p.match("python")
```
위 코드를 축약해 다음과 같이 사용할 수 있다.
``` python
>>> m = re.match('[a-z]+', "python")
```
  
한 번 만든 패턴 객체를 여러번 사용해야할 때는 위의 방법을 사용하는 것이 편하다.


##### 컴파일 옵션
* DOTALL(S) - `.` 이 줄바꿈 문자를 포함하여 모든 문자와 매치할 수 있도록 한다.
* IGNORECASE(I) - 대소문자에 관계없이 매치할 수 있도록 한다.
* MULTILINE(M) - 여러줄과 매치할 수 있도록 한다. (`^`, `$` 메타문자의 사용과 관계가 있는 옵션이다)
* VERBOSE(X) - verbose 모드를 사용할 수 있도록 한다. (정규식을 보기 편하게 만들수 있고 주석등을 사용할 수 있게된다.)

**DOTALL, S**  
`\n`문자도 포함하여 매치하고 싶을 때, `re.DOTALL` 또는 `re.S`옵션을 사용해 정규식을 컴파일한다.

In [19]:
import re
p = re.compile('a.b')
m = p.match('a\nb')
print(m)

None


In [21]:
p = re.compile('a.b', re.DOTALL)
m = p.match('a\nb')
print(m)

#여러 줄로 이루어진 문자열에서 \n에 상관없이 검색할 때 많이 사용한다.

<re.Match object; span=(0, 3), match='a\nb'>


**IGNORECASE, I**  
대소문자 구별없이 매치를 수행할 때 사용한다.

In [22]:
p = re.compile('[a-z]+', re.I) # 소문자만을 의미하는 정규식
p.match("python")

<re.Match object; span=(0, 6), match='python'>

In [23]:
p.match("PYTHON")

<re.Match object; span=(0, 6), match='PYTHON'>

**MULTILINE, M**  
`^` : 문자열의 처음을 의미  
`$` : 문자열의 마지막을 의미

In [25]:
import re
p = re.compile("^python\s\w+") 
# 'python'이라는 문자열로 시작하고, 그 뒤에 whitespace, 그 뒤에 단어가 와야한다는 의미


data = """python one
life is too short
python two
you need python
python three"""

print(p.findall(data))

# ^ 메타문자에 의해 python 문자열을 사용한 첫 번째 줄만 매치된다.

['python one']


In [26]:
import re
p = re.compile("^python\s\w+", re.MULTILINE) #각 라인의 처음으로 인식

data = """python one
life is too short
python two
you need python
python three"""

print(p.findall(data))

['python one', 'python two', 'python three']


**VERBOSE, X**  
정규식을 주석 또는 줄 단위로 구분

In [27]:
charref = re.compile(r'&[#](0[0-7]+|[0-9]+|x[0-9a-fA-F]+);')

In [28]:
charref = re.compile(r"""
 &[#]                # Start of a numeric entity reference
 (
     0[0-7]+         # Octal form
   | [0-9]+          # Decimal form
   | x[0-9a-fA-F]+   # Hexadecimal form
 )
 ;                   # Trailing semicolon
""", re.VERBOSE)
# 문자열에 사용된 whitespace는 컴파일할 때 제거된다.
# 또한 줄단위로 주석문을 작성할 수 있다.

**백슬래시 문제**  
`\section`문자열을 찾기 위한 정규식을 만들 때,  
\s 가 whitespace로 해석되기 때문에 `[ \t\n\r\f\v]ection` 이와 같은 의미이다.   
의도한대로 매치하고 싶다면 `\\section`과 같이 변경해야한다.(이스케이프 처리)

In [30]:
p = re.compile('\\section')
# 파이썬 정규식 엔진에서, 
# 문자열 리터럴 규칙에 따라 \\이 \로 변경되는 문제 발생

In [31]:
p = re.compile('\\\\section')
p = re.compile(r'\\section') #raw string규칙

## 7-3. 강력한 정규 표현식의 세계로
##### 메타문자
앞에서 배운 메타문자들은 매치가 진행될 때 현재 매치되고 있는 문자열의 위치가 변경(소비)된다.  
이와 달리 문자열을 소비시키지 않는(소비가 없는)메타 문자가 있다.

In [32]:
# | : '또는(or)' 이라는 의미

p = re.compile('Crow|Servo')
m = p.match('CrowHello')
print(m)

<re.Match object; span=(0, 4), match='Crow'>


In [33]:
# ^ : 문자열의 맨 처음과 일치함을 의미

print(re.search('^Life', "Life is too short"))
print(re.search('^Life', "My Life"))

<re.Match object; span=(0, 4), match='Life'>
None


In [34]:
# $ : 문자열의 끝과 매치함을 의미

print(re.search('short$', "Life is too short"))
print(re.search('short$', 'Life is too short, you need python'))

# ^ 또는 $ 문자를 메타 문자가 아닌 문자 그 자체로 매치하고 싶은 경우에는 
# \^, \$ 로 사용하면 된다.

<re.Match object; span=(12, 17), match='short'>
None


In [35]:
# \A : 문자열의 처음과 매치됨을 의미
# (re.MULTILINE 옵션을 사용할 경우 줄과 상관없이 전체 문자열의 처음하고만 매치된다.)

# \Z : 문자열의 끝과 매치
# (re.MULTILINE 옵션을 사용할 경우 줄과 상관없이 전체 문자열의 끝과 매치된다.)

In [37]:
# \b : 단어 구분자이다.

p = re.compile(r'\bclass\b')
print(p.search('no class at all'))
print(p.search('the declassified algorithm'))
print(p.search('one subclass is'))

<re.Match object; span=(3, 8), match='class'>
None
None


In [38]:
# \B : \b 메타 문자와 반대의 경우, whitespace로 구분된 단어가 아닌경우에만 매치된다.

p = re.compile(r'\Bclass\B')
print(p.search('no class at all'))
print(p.search('the declassified algorithm'))
print(p.search('one subclass is'))

None
<re.Match object; span=(6, 11), match='class'>
None


##### 그루핑

그룹을 만들어 주는 메타문자는 `()`이다.

In [39]:
p = re.compile('(ABC)+')
m = p.search('ABCABCABC OK?')
print(m)
print(m.group())

<re.Match object; span=(0, 9), match='ABCABCABC'>
ABCABCABC


In [43]:
# p = re.compile(r"\w+\s+\d+[-]\d+[-]\d+")
# m = p.search("park 010-1234-1234")
p = re.compile(r"(\w+)\s+\d+[-]\d+[-]\d+")
m = p.search("park 010-1234-1234")
print(m.group(1)) #이름
p = re.compile(r"(\w+)\s+(\d+[-]\d+[-]\d+)")
m = p.search("park 010-1234-1234")
print(m.group(2)) #전화번호
p = re.compile(r"(\w+)\s+((\d+)[-]\d+[-]\d+)")
m = p.search("park 010-1234-1234")
print(m.group(3)) #국번

park
010-1234-1234
010


> 그룹이 중첩되어 있는 경우는 바깥쪽부터 시작해 안쪽으로 들어갈수록 인덱스가 증가한다.
  
     
group(인덱스)|	설명
-------------|-------------
group(0)	 |매치된 전체 문자열
group(1)	 |첫 번째 그룹에 해당되는 문자열
group(2)	 |두 번째 그룹에 해당되는 문자열
group(n)	 |n 번째 그룹에 해당되는 문자열

**그루핑된 문자열 재참조하기**


In [44]:
p = re.compile(r'(\b\w+)\s+\1') 
# 2개의 동일한 단어를 연속적으로 사용해야만 매치
# \1은 정규식의 그룹 중 첫 번째 그룹을 의미
p.search('Paris in the the spring').group()

'the the'

**그루핑된 문자열에 이름 붙이기**   


```(?P<name>\w+)\s+((\d+)[-]\d+[-]\d+)```  
```(?P<그룹명>...)``` 과 같은 확장 구문을 사용해야한다.  
재참조할 때에는 ```(?P=그룹이름)```이라는 확장 구문을 사용한다.


In [45]:
p = re.compile(r"(?P<name>\w+)\s+((\d+)[-]\d+[-]\d+)")
m = p.search("park 010-1234-1234")
print(m.group("name"))

park


In [46]:
p = re.compile(r'(?P<word>\b\w+)\s+(?P=word)') 
#정규식 안에서 재참조 하는 것도 가능
p.search('Paris in the the spring').group()

'the the'

##### 전방 탐색
* 긍정형 전방 탐색((?=...)) - ... 에 해당되는 정규식과 매치되어야 하며 조건이 통과되어도 문자열이 소비되지 않는다.
* 부정형 전방 탐색((?!...)) - ...에 해당되는 정규식과 매치되지 않아야 하며 조건이 통과되어도 문자열이 소비되지 않는다.

In [47]:
p = re.compile(".+:")
m = p.search("http://google.com")
print(m.group())

http:


**긍정형 전방 탐색**  


In [48]:
p = re.compile(".+(?=:)")
# : 에 해당하는 문자열이 검색에는 포함되지만 검색결과에는 제외되어 :제거 후 돌려준다.
m = p.search("http://google.com")
print(m.group())

http


**부정형 전방 탐색**
`.*[.].*$` : 파일이름 + . + 확장자  
확장자가 bat인 파일은 제외해야 한다 는 조건 추가
`.*[.]([^b].?.?|.[^a]?.?|..?[^t]?)$`
여기서 exe 파일도 제외하라는 조건이 추가
 
==> `.*[.](?!bat$).*$`: 확장자가 bat가 아닌 경우에만 통과    
`.*[.](?!bat$|exe$).*$`